# Title

In [1]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from types import MethodType

In [3]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print(
            "Property.__init__:",
            f"{self=}",
            f"{fget=}",
            f"{fset=}",
            f"{fdel=}",
            f"{doc=}",
            sep="\n\t",
        )
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        print("Property.__get__:", f"{self=}", f"{obj=}", f"{objtype=}", sep="\n\t")
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        if obj is None:
            result = self
        else:
            result = self.fget(obj)
        print(f">>> Property.__get__ ", f"{(obj is None)=}", f"{result=}", sep="\n\t")
        return result

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [4]:
class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        print(f"ClassMethod.__init__:", f"{self=}", f"{f=}", sep="\n\t")
        self.f = f

    def __get__(self, obj, cls=None):
        print(f"ClassMethod.__get__:", f"{self=}", f"{obj=}", f"{cls=}", sep="\n\t")
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.f), "__get__"):
            # result = self.f.__get__(cls)
            result = MethodType(self.f.__get__, cls).__call__()
        else:
            result = MethodType(self.f, cls)
        print(
            f">>> ClassMethod.__get__:",
            f"{hasattr(type(self.f), '__get__')=}",
            f"{result=}",
            sep="\n\t",
        )
        return result

In [5]:
class ClassPropertyMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        print(
            "ClassPropertyMethod.__init__:",
            f"{self=}",
            f"{fget=}",
            f"{fset=}",
            f"{fdel=}",
            f"{doc=}",
            sep="\n\t",
        )
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, cls=None):
        print(
            f"ClassPropertyMethod.__get__:",
            f"{self=}",
            f"{obj=}",
            f"{cls=}",
            sep="\n\t",
        )
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.fget), "__get__"):
            # result = self.f.__get__(cls)
            result = MethodType(self.fget.__get__, cls).__call__()
        else:
            result = MethodType(self.fget, cls)
        print(
            f">>> ClassPropertyMethod.__get__:",
            f"{hasattr(type(self.fget), '__get__')=}",
            f"{result=}",
            sep="\n\t",
        )
        return result

    def __set__(self, obj, value):
        if self.fset.fset is None:
            raise AttributeError("can't set attribute")
        self.fset.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [26]:
class ClassProperty(property):

    #     def __init__(self, fget=None, fset=None, fdel=None, doc=None):
    #         print("ClassProperty.__init__:", f"{self=}", f"{fget=}", f"{fset=}", f"{fdel=}", f"{doc=}", sep="\n\t")
    #         self.fget = fget #if fget is None else property(fget) if fget is not None else fget
    #         self.fset = fset #if fset is None else property(fset) if fset is not None else fset
    #         self.fdel = fdel #if fdel is None else property(fdel) if fdel is not None else fdel
    #         if doc is None and fget is not None:
    #             doc = fget.__doc__
    #         self.__doc__ = doc

    def __get__(self, obj, cls=None):
        print(f"ClassProperty.__get__:", f"{self=}", f"{obj=}", f"{cls=}", sep="\n\t")
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.fget), "__get__"):
            # result = self.f.__get__(cls)
            result = MethodType(self.fget.__get__, cls).__call__()
        else:
            result = MethodType(self.fget, cls)
        print(
            f">>> ClassProperty.__get__:",
            f"{hasattr(type(self.fget), '__get__')=}",
            f"{result=}",
            sep="\n\t",
        )
        return property(result).__get__(obj)

In [27]:
class Testit:
    @classmethod
    @property
    def clsprop(cls):
        """Real Class-Property"""
        return f"My name is {cls.__name__}"

    @ClassMethod
    @Property
    def myclsprop(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"

    @ClassPropertyMethod
    @Property
    def myclsprop2(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"

    @ClassProperty
    def myclsprop3(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"

    @property
    def prop(self):
        return 42


Testit.myclsprop, Testit.myclsprop2, Testit.myclsprop3

In [19]:
Testit.myclsprop3

In [105]:
sorted(Testit.__dict__["prop"].__dir__())

In [106]:
sorted(Testit.__dict__["myclsprop3"].__dir__())